In [89]:
import pandas as pd
import numpy as np

# 数据预处理
data=pd.read_csv('train1.csv',encoding='big5')
data=data.iloc[:,3:]
data[data=="NR"]=0
np_data=data.to_numpy()

# 将每个月20天的数据水平拼接 做成12个这样的字典
# 12 * 18*480 12个18*480的数据块
month_data={}
for i in range(12):
    block=np.empty((18,480))
    for j in range(20):
        block[:,j*24:(j+1)*24]=np_data[18*(i*20+j):18*(i*20+j+1),:]
    month_data[i]=block

# 每10小时作为一条记录,前9个小时用作x,第10小时用作y
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x.shape)
print(y.shape)

# 标准化 使得数据的均值为0,标准差是1 (X-mean)/std
mean = np.mean(x,axis=0) # dim 18*9
std = np.std(x,axis=0)  #dim 18*9
print(mean.shape)

for col in range(x.shape[1]):
    if std[col]!=0:
        x[:,col]=(x[:,col]-mean[col])/std[col]
# print(x)

# 训练集再划分
import math as mt
# 训练模型用
x_train=x[:mt.floor(x.shape[0]*0.8),:]
y_train=y[:mt.floor(y.shape[0]*0.8),:]
# 评估训练模型好坏
x_val=x[mt.floor(x.shape[0]*0.8):,:]
y_val=y[mt.floor(y.shape[0]*0.8):,:]
print('*'*100)

# 训练模型
dim=18*9+1 # 包含常数项
w = np.zeros([dim,1])
x=np.concatenate((np.ones([12*471,1]),x),axis=1).astype('float')
learning_rate=100
iter_time=10000
adagrad=np.zeros([dim,1])
eps=0.0000000001
# 这里所有的训练集数据都用到了
for i in range(iter_time):
    loss=np.sqrt(np.sum(np.power(np.dot(x,w)-y,2),axis=0)/12/471)
    if(i%100==0):
        print(str(i) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)

# 测试集数据预处理
# testdata = pd.read_csv('./test1.csv', header = None, encoding = 'big5')
# test_data = testdata.iloc[:, 2:]
# test_data[test_data == 'NR'] = 0
# test_data = test_data.to_numpy()
# test_x = np.empty([240, 18*9], dtype = float)
# for i in range(test_x.shape[1]):
#     if std_x[i] != 0:
#         test_x[:,i] = (test_x[:,i] - mean[i]) / std[i]
# test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x = np.concatenate((np.ones([x.shape[0]-mt.floor(x.shape[0]*0.8), 1]), x_val), axis = 1).astype(float)

# 预测
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
print('*'*100)
err=np.ones([x.shape[0]-mt.floor(x.shape[0]*0.8),1])-(np.abs(y_val-ans_y)/y_val)
print(err)
# 保存预测结果
# import csv
# with open('submit.csv', mode='w', newline='') as submit_file:
#     csv_writer = csv.writer(submit_file)
#     header = ['id', 'value']
#     print(header)
#     csv_writer.writerow(header)
#     for i in range(240):
#         row = ['id_' + str(i), ans_y[i][0]]
#         csv_writer.writerow(row)
#         print(row)

(5652, 162)
(5652, 1)
(162,)
****************************************************************************************************
0:[27.07121483]
100:[33.7890586]
200:[19.9137513]
300:[13.53106819]
400:[10.64546616]
500:[9.27735346]
600:[8.51804205]
700:[8.01406199]
800:[7.63675682]
900:[7.33656374]
1000:[7.09096864]
1100:[6.88731148]
1200:[6.7171163]
1300:[6.57410212]
1400:[6.45338117]
1500:[6.35106247]
1600:[6.26401229]
1700:[6.18968838]
1800:[6.12601676]
1900:[6.07129714]
2000:[6.02412917]
2100:[5.98335456]
2200:[5.94801117]
2300:[5.91729666]
2400:[5.89053938]
2500:[5.86717504]
2600:[5.84672795]
2700:[5.82879577]
2800:[5.81303717]
2900:[5.79916169]
3000:[5.78692146]
3100:[5.77610441]
3200:[5.76652861]
3300:[5.75803759]
3400:[5.75049647]
3500:[5.74378871]
3600:[5.73781339]
3700:[5.73248291]
3800:[5.72772112]
3900:[5.72346166]
4000:[5.71964659]
4100:[5.71622525]
4200:[5.71315322]
4300:[5.71039156]
4400:[5.707906]
4500:[5.70566639]
4600:[5.70364613]
4700:[5.70182172]
4800:[5.70017239]


E:\anaconda\envs\scikit-learn\lib\site-packages\ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in true_divide
